In [ ]:
import numpy as np
import pandas as pd

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
import glob
Mild = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented/*.*')
Moderate = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/ModerateDemented/*.*')
NonDemented = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/NonDemented/*.*')
VeryMild = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented/*.*')


data=[]
labels = []

for i in NonDemented:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data.append(image)
    labels.append(0)

for i in VeryMild:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data.append(image)
    labels.append(1)

for i in Mild:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data.append(image)
    labels.append(2)

for i in Moderate:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data.append(image)
    labels.append(3)

In [ ]:
import random
Moderate_US = Moderate
Mild_US = random.sample(Mild, 52)
VeryMild_US = random.sample(VeryMild, 52)
NonDemented_US = random.sample(NonDemented, 52)

In [ ]:
from PIL import Image

data1 =[]
labels1 = []

for i in NonDemented_US: 
    image=tf.keras.preprocessing.image.load_img(i,target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data1.append(image)
    labels1.append(0)
    
for i in VeryMild_US:   
    image=tf.keras.preprocessing.image.load_img(i,target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data1.append(image)
    labels1.append(1)

for i in Mild_US:   
    image=tf.keras.preprocessing.image.load_img(i,target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data1.append(image)
    labels1.append(2)

for i in Moderate_US:   
    image=tf.keras.preprocessing.image.load_img(i,target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data1.append(image)
    labels1.append(3)

In [ ]:
import glob
Mild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented/*.*')
Moderate_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented/*.*')
NonDemented_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented/*.*')
VeryMild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented/*.*')


data_test = []
labels_test  = []

for i in NonDemented_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_test.append(image)
    labels_test.append(0)

for i in VeryMild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_test.append(image)
    labels_test.append(1)

for i in Mild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_test.append(image)
    labels_test.append(2)

for i in Moderate_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_test.append(image)
    labels_test.append(3)

In [ ]:
data1 = np.array(data1)
labels1 = np.array(labels1)

data_test = np.array(data_test)
labels_test = np.array(labels_test)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest'
    )

train_data_gen = datagen.flow(
    x=data1,
    y=labels1,
    batch_size=32,
)

In [ ]:
test_data_gen = datagen.flow(
    x=data_test,
    y=labels_test
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
from keras.callbacks import Callback

class MetricsCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        x_val, y_val = self.validation_data
        y_pred = np.argmax(self.model.predict(x_val), axis=1)

        true_positives = np.sum(np.logical_and(y_val == 1, y_pred == 1))
        false_positives = np.sum(np.logical_and(y_val == 0, y_pred == 1))

        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / np.sum(y_val == 1) if np.sum(y_val == 1) > 0 else 0

        print(f'\nPrecision: {precision:.4f} - Recall: {recall:.4f}')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),

    Dense(128, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),

    Dense(4, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data_gen.x,train_data_gen.y ,batch_size=16, epochs=10,validation_data=test_data_gen, callbacks=[MetricsCallback((test_data_gen.x, test_data_gen.y))])
        